In [1]:
import requests
import numpy as np
import pandas as pd

### Calling lat, lon from csv

In [2]:
stations_df=pd.read_csv('../../raw_data/stations_df_st.csv')

In [3]:
stations_df

,Unnamed: 0,s_id,Station_name,Street,Neighbourhood,Area_loc,s_lat,s_lon,s_num_docks,Colours,index,Size_bucket
0,0,1023,"River Street , Clerkenwell",River Street,Clerkenwell,North,51.529163,-0.109970,19,#ffbaba,0,100
1,1,1018,"Phillimore Gardens, Kensington",Phillimore Gardens,Kensington,South_West,51.499606,-0.197574,36,#ff7b7b,1,150
2,2,1012,"Christopher Street, Liverpool Street",Christopher Street,Liverpool Street,East,51.521283,-0.084605,32,#ff5252,2,150
3,3,1013,"St. Chad's Street, King's Cross",St. Chad's Street,King's Cross,North,51.530059,-0.120973,23,#ffbaba,0,100
4,4,3420,"Sedding Street, Sloane Square",Sedding Street,Sloane Square,South_West,51.493130,-0.156876,27,#ff7b7b,1,100
...,...,...,...,...,...,...,...,...,...,...,...,...
791,791,300062,"South Bermondsey Station, Bermondsey",South Bermondsey Station,Bermondsey,East,51.488067,-0.057774,23,#ff5252,2,100
792,792,300061,"Crimscott Street, Bermondsey",Crimscott Street,Bermondsey,East,51.495598,-0.078893,18,#ff5252,2,100
793,793,300060,"Brandon Street, Walworth",Brandon Street,Walworth,East,51.489102,-0.091548,21,#ff5252,2,100
794,794,300059,"The Blue, Bermondsey",The Blue,Bermondsey,East,51.492221,-0.062513,21,#ff5252,2,100


In [4]:
origin='Shoreditch Park, Hoxton'
destination='Finsbury Circus, Liverpool Street'

In [108]:
origin_lat=stations_df[stations_df['Station_name']==origin]['s_lat'].tolist()[0]
origin_lon=stations_df[stations_df['Station_name']==origin]['s_lon'].tolist()[0]
destination_lat=stations_df[stations_df['Station_name']==destination]['s_lat'].tolist()[0]
destination_lon=stations_df[stations_df['Station_name']==destination]['s_lon'].tolist()[0]


In [119]:
origin_lat,origin_lon

(51.534042, -0.086379)

In [120]:
destination_lat,destination_lon

(51.517075, -0.086685)

In [241]:
parameters={'mode':'cycle','cyclepreference':'AllTheWay'}


In [237]:
url=f'https://api.tfl.gov.uk/Journey/JourneyResults/{origin_lat},{origin_lon}/to/{destination_lat},{destination_lon}'
# url2=f'https://api.tfl.gov.uk/Journey/JourneyResults/N15DY/to/{destination_lat,destination_lon}'

In [238]:
response= requests.get(url,params=parameters)

In [239]:
journey=response.json()

In [240]:
journey

{'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.ItineraryResult, Tfl.Api.Presentation.Entities',
 'journeys': [{'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.Journey, Tfl.Api.Presentation.Entities',
   'startDateTime': '2023-09-04T16:58:00',
   'duration': 12,
   'arrivalDateTime': '2023-09-04T17:10:00',
   'alternativeRoute': False,
   'legs': [{'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.Leg, Tfl.Api.Presentation.Entities',
     'duration': 12,
     'speed': 'default',
     'instruction': {'$type': 'Tfl.Api.Presentation.Entities.Instruction, Tfl.Api.Presentation.Entities',
      'summary': 'Cycle to 1 Circus Place',
      'detailed': 'Cycle to 1 Circus Place',
      'steps': [{'$type': 'Tfl.Api.Presentation.Entities.InstructionStep, Tfl.Api.Presentation.Entities',
        'description': ' for 35 metres',
        'turnDirection': 'STRAIGHT',
        'streetName': '',
        'distance': 35,
        'cumulativeDistance': 35,
        'skyDirection': 141,
    

In [263]:
duration_cycle=journey['journeys'][0]['legs'][0]['duration']
duration_cycle

12

In [189]:
steps=journey['journeys'][0]['legs'][0]['instruction']['steps']

In [190]:
steps_lat=[origin_lat]
steps_lon=[origin_lon]
for step in steps:
    steps_lat.append(step['latitude'])
    steps_lon.append(step['longitude'])

steps_lat.append(destination_lat)
steps_lon.append(destination_lon)


In [191]:
##

In [196]:
steps_lat_extra=[]
steps_lon_extra=[]
for i in range(0,len(steps_lat)-1):
   steps_lat_extra+=list(np.linspace(steps_lat[i-1],steps_lat[i],30))
   steps_lon_extra+=list(np.linspace(steps_lon[i-1],steps_lon[i],30))


In [225]:
itinerary_df=pd.DataFrame()
itinerary_df['lat']=steps_lat_extra
itinerary_df['lon']=steps_lon_extra

In [226]:
# itinerary_df.iloc[0]['size']=100
# # itinerary_df.loc[-1]['size']=100
# itinerary_df



In [230]:
size_iti=[100]
for i in range(itinerary_df.shape[0]-2):
    size_iti+=[25]
size_iti+=[100]

In [231]:
itinerary_df['size']=size_iti
itinerary_df

,lat,lon,size
0,51.517075,-0.086685,100
1,51.517660,-0.086674,25
2,51.518245,-0.086664,25
3,51.518830,-0.086653,25
4,51.519415,-0.086643,25
...,...,...,...
655,51.517501,-0.086841,25
656,51.517476,-0.086784,25
657,51.517450,-0.086726,25
658,51.517425,-0.086668,25


In [233]:
color_iti=['#ec0000']
for i in range(itinerary_df.shape[0]-2):
    color_iti+=['#333333']
color_iti+=['#ec0000']

In [234]:
itinerary_df['color']=color_iti

## API: Walking vs Cycling

In [271]:
parameters={'mode':'cycle,walking,bus'}

In [272]:
url=f'https://api.tfl.gov.uk/Journey/JourneyResults/{origin_lat},{origin_lon}/to/{destination_lat},{destination_lon}'
# url2=f'https://api.tfl.gov.uk/Journey/JourneyResults/N15DY/to/{destination_lat,destination_lon}'

In [273]:
response_2= requests.get(url,params=parameters)

In [274]:
journey_2=response_2.json()

In [277]:
journey_2

{'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.ItineraryResult, Tfl.Api.Presentation.Entities',
 'journeys': [{'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.Journey, Tfl.Api.Presentation.Entities',
   'startDateTime': '2023-09-04T17:21:00',
   'duration': 21,
   'arrivalDateTime': '2023-09-04T17:42:00',
   'alternativeRoute': False,
   'legs': [{'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.Leg, Tfl.Api.Presentation.Entities',
     'duration': 4,
     'instruction': {'$type': 'Tfl.Api.Presentation.Entities.Instruction, Tfl.Api.Presentation.Entities',
      'summary': 'Walk to Mintern Street',
      'detailed': 'Walk to Mintern Street',
      'steps': [{'$type': 'Tfl.Api.Presentation.Entities.InstructionStep, Tfl.Api.Presentation.Entities',
        'description': ' for 35 metres',
        'turnDirection': 'STRAIGHT',
        'streetName': '',
        'distance': 35,
        'cumulativeDistance': 35,
        'skyDirection': 141,
        'skyDirectionDescription

In [278]:
duration_walking=journey_2['journeys'][0]['legs'][0]['duration']
duration_walking

4

In [279]:
duration_walking=journey_2['journeys'][0]['legs'][0]['duration']
duration_walking

4